<a href="https://colab.research.google.com/github/takatakamanbou/Vision/blob/main/Vision2023_report03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision2023-report03





課題の期限や提出の方法などについては，別途お知らせしています



In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

from scipy.stats import norm
from scipy.stats import multivariate_normal

from sklearn.mixture import GaussianMixture
from sklearn.datasets import fetch_openml
from sklearn.datasets import make_moons

---
## 問題1
---

以下では，2次元3クラスのデータを対象としてある実験を行っている．
コードセルを順に実行し，問に答えなさい．
解答欄に文章を記入することを求めている箇所がある．その場合，次のセルの内容を参考にしたらよい．


※ Colab の Markdown セルには，LaTeX と呼ばれる組版ソフトウェアの表記法で数式を書くことができます（ *LaTeX* をキーワードにしてウェブで検索したらいっぱい情報あります）．このセルをダブルクリックしてソースを表示させるとわかります．

$p(\mathbf{x})$

$p(y)$

$p(\mathbf{x}|y)$

$p(y|\mathbf{x})$

$y = 1, 2, 3$

In [ ]:
# 2次元3クラスのデータ
df = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/2dim3class2.csv', header=0)
X = df.loc[:, ['x1', 'x2']].to_numpy()
N, D = X.shape
Y = df['label'].to_numpy()
K = 3
print(f'N = {N}, D = {D}, K = {K}')

# 散布図を描く
fig, ax = plt.subplots()
for k in range(K):
    ax.scatter(X[Y==k, 0], X[Y==k, 1], label=f'Class{k}')
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
ax.set_aspect('equal')
ax.legend()
plt.show()

#### Q1.1 次のコードセルは何を計算しているか

解答欄に数式も用いて説明を書きなさい．

In [ ]:
P = np.empty(K)
for k in range(K):
    P[k] = np.sum(Y==k)/N
print('P = ', P)

【Q1.1 解答欄】



#### Q1.2 次のコードセルは何を計算しているか

解答欄に数式も用いて説明を書きなさい．

In [ ]:
mu = np.empty((K, D))
cov = np.empty((K, D, D))
for k in range(K):
    XX = X[Y==k, :]
    mu[k] = np.mean(XX, axis=0)
    cov[k] = (XX - mu[k]).T @ (XX - mu[k]) / XX.shape[0]
    print(f'### Class{k}')
    print(mu[k])
    print(cov[k])
    print()

【Q1.2 解答欄】



#### Q1.3 次のコードセルの `## ここから ##` と `## ここまで ##` の間のコードは何を求めているか．

解答欄に数式も用いて説明を書きなさい．

cf. `multivariate_normal.logpdf` についてはこちら: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.multivariate_normal.html

In [ ]:
# データを識別
hoge = np.empty(K)
ncorrect = 0
for n in range(len(X)):
    ## ここから ##
    for k in range(K):
        hoge[k] = np.log(P[k]) + multivariate_normal.logpdf(X[n], mean=mu[k], cov=cov[k])
    yp = np.argmax(hoge)
    ## ここまで ##
    if yp == Y[n]:
        ncorrect += 1
print(f'{ncorrect}/{len(X)} = {ncorrect/len(X):.3f}')

【Q1.3 解答欄】



#### Q1.4 次のコードセルを実行すると得られる二つの図はそれぞれ何を表しているか

解答欄に説明を書きなさい．数式を用いてもよい．

In [ ]:
# グラフ描画用のグリッドデータの作成
xmin, xmax = -5, 5
ymin, ymax = -5, 5
x_mesh, y_mesh = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X_mesh = np.dstack((x_mesh, y_mesh))

# X_mesh の各点における〇〇確率の推定
p = np.empty((K, X_mesh.shape[0]*X_mesh.shape[1]))
for k in range(K):
    p[k, :] = P[k] * multivariate_normal.pdf(X_mesh, mean=mu[k], cov=cov[k]).reshape(-1)
p /= np.sum(p, axis=0)
pp = p.reshape((K, X_mesh.shape[0], X_mesh.shape[1]))

# グラフ
fig = plt.figure(figsize=(9, 6))

# 左図
ax0 = fig.add_subplot(121)
for k in range(K):
    ax0.scatter(X[Y==k, 0], X[Y==k, 1])
    ax0.contour(x_mesh, y_mesh, multivariate_normal.pdf(X_mesh, mean=mu[k], cov=cov[k]), levels=4)
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')

# 右図
cmap = ['Blues', 'Oranges', 'Greens']
ax1 = fig.add_subplot(122)
for k in range(K):
    ax1.scatter(X[Y==k, 0], X[Y==k, 1])
    ax1.contourf(x_mesh, y_mesh, pp[k], levels=[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')

plt.show()

【Q1.4 解答欄】



---
## 問題2
---





#### Q2 **混合正規分布モデル** (Gaussian Mixture Model) について調べた上で，次のことをやりなさい

次の式の右辺に現れる記号がそれぞれ何を表すのかを解答欄に簡単に書きなさい．

$$
p(\mathbf{x}) = \sum_{m=1}^{M} w_m N(\mathbf{x}|\mathbf{\mu}_m, \Sigma_m)
$$


【Q2 解答欄】



---
## 問題3
---

以下では，2次元2クラスのデータに混合正規分布を当てはめて識別する実験を行っている．
コードセルを順に実行し，問に答えなさい．
解答欄に文章を記入することを求めている箇所がある．数式も用いたい場合は問題1や以下を参考にしたらよい．


データの準備．

In [ ]:
moonX, moonY = make_moons(n_samples=(800, 200), noise=0.25)
NL, NT = 500, 500
XL, YL = moonX[:NL, :], moonY[:NL]
XT, YT = moonX[NL:, :], moonY[NL:]
print(XL.shape, YL.shape)
print(XT.shape, YT.shape)

In [ ]:
fig = plt.figure(figsize=(9, 6))
ax0 = fig.add_subplot(121)
ax0.scatter(XL[YL==0, 0], XL[YL==0, 1], marker='.', label='Class0')
ax0.scatter(XL[YL==1, 0], XL[YL==1, 1], marker='.', label='Class1')
ax0.set_xlim(-1.5, 2.5)
ax0.set_ylim(-2, 2)
ax0.set_aspect('equal')
ax0.legend()
ax0.set_title('training data')


ax1 = fig.add_subplot(122)
ax1.scatter(XT[YT==0, 0], XT[YT==0, 1], marker='.', label='Class0')
ax1.scatter(XT[YT==1, 0], XT[YT==1, 1], marker='.', label='Class1')
ax1.set_xlim(-1.5, 2.5)
ax1.set_ylim(-2, 2)
ax1.set_aspect('equal')
ax1.legend()
ax1.set_title('test data')

plt.show()

#### Q3 次の2つのコードセルを，`M` の値をいろいろ変えながら実行し，結果を観察しなさい．

解答欄に，どのような実験結果が得られたか，および，それに対する考察を書きなさい．混合正規分布モデルの学習は，パラメータの初期値が異なれば異なる結果となり得ることに注意（同じモデルでも何回か実行してみるのがよいでしょう）．




In [ ]:
K = 2
N, D = XL.shape
M = 1

# 〇〇確率を推定
P = np.empty(K)
for k in range(K):
    P[k] = np.sum(YL==k)/N
print('P =', P)

# 〇〇を混合正規分布でモデル化
mu = np.empty((K, M, D))
cov = np.empty((K, M, D, D))
model = np.empty(K, dtype=object)
for k in range(K):
    XX = XL[YL==k, :]
    model[k] = GaussianMixture(n_components=M, covariance_type='full')
    model[k].fit(XX)
    mu[k, ::] = model[k].means_
    cov[k, ::] = model[k].covariances_

# 学習データの識別
N = XL.shape[0]
LL = np.empty((N, K))
for n in range(N):
    for k in range(K):
        LL[n, k] = np.log(P[k]) + model[k].score_samples(XL[n, np.newaxis, :])
Yp = np.argmax(LL, axis=1)
ncorrect = np.sum(YL == Yp)
print(f'学習データ: {ncorrect/N}')

# テストデータの識別
N = XT.shape[0]
LL = np.empty((N, K))
for n in range(N):
    for k in range(K):
        LL[n, k] = np.log(P[k]) + model[k].score_samples(XT[n, np.newaxis, :])
Yp = np.argmax(LL, axis=1)
ncorrect = np.sum(YT == Yp)
print(f'テストデータ: {ncorrect/N}')

In [ ]:
# グラフ描画用のグリッドデータの作成
xmin, xmax = -1.5, 2.5
ymin, ymax = -2, 2
x_mesh, y_mesh = np.mgrid[xmin:xmax:0.02, ymin:ymax:0.02]
X_mesh = np.dstack((x_mesh, y_mesh))
print(X_mesh.shape)

# 事後確率の推定
p = np.empty((K, X_mesh.shape[0]*X_mesh.shape[1]))
for k in range(K):
    proba = np.exp(model[k].score_samples(X_mesh.reshape((-1, 2))))
    p[k, :] = P[k] * proba
p /= np.sum(p, axis=0)
pp = p.reshape((K, X_mesh.shape[0], X_mesh.shape[1]))

# グラフ
fig = plt.figure(facecolor="white", figsize=(12, 8))

# 左図
ax0 = fig.add_subplot(121)
for k in range(K):
    ax0.scatter(XL[YL==k, 0], XL[YL==k, 1])
    for m in range(M):
        z = multivariate_normal.pdf(X_mesh, mean=mu[k, m], cov=cov[k, m])
        ax0.contour(x_mesh, y_mesh, z, levels=4)
ax0.set_xlim(xmin, xmax)
ax0.set_ylim(ymin, ymax)
ax0.set_aspect('equal')

# 右図
cmap = ['Blues', 'Oranges', 'Greens']
ax1 = fig.add_subplot(122)
for k in range(K):
    ax1.scatter(XL[YL==k, 0], XL[YL==k, 1])
    ax1.contourf(x_mesh, y_mesh, pp[k], levels=[0.5, 0.6, 0.7, 0.8, 0.9, 1.0], cmap=cmap[k], alpha=0.3)
ax1.set_xlim(xmin, xmax)
ax1.set_ylim(ymin, ymax)
ax1.set_aspect('equal')

plt.show()

【Q3 解答欄】



---
## 問題4
---

以下では，MNISTと呼ばれる手書き数字データを対象として，ある実験を行っている．
コードセルを順に実行し，問に答えなさい．



### 準備

In [ ]:
# データを画像として表示するための関数
#
def display(data, nx, ny, nrow=28, ncol=28, gap=4):

    assert data.shape[0] == nx*ny
    assert data.shape[1] == nrow*ncol

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = data[iy*nx+ix].reshape((nrow, ncol))

    # 画像の出力
    plt.axis('off')
    plt.imshow(img, cmap = 'gray')
    plt.show()

In [ ]:
# MNIST のデータを入手 https://www.openml.org/d/554
X_mnist, Y_mnist = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False, parser='pandas')
Y_mnist = Y_mnist.astype(int)

# 最初の6万枚を学習データ，残り1万枚をテストデータとするのが一般的だが，
# ここでは計算コストを減らすため学習データ数を削減
XL, YL = X_mnist[:10000, :], Y_mnist[:10000]
XT, YT = X_mnist[60000:, :], Y_mnist[60000:]
print(XL.shape, YL.shape, XT.shape, YT.shape)

In [ ]:
# 一部の可視化
nx, ny = 10, 5
display(XL[:ny*nx], nx, ny)
for iy in range(ny):
    print(YL[iy*nx:(iy+1)*nx])

### 実験

In [ ]:
K = 10
N, D = XL.shape
M = 16

# 〇〇確率を推定
P = np.empty(K)
for k in range(K):
    P[k] = np.sum(YL==k)/N
print('P =', P)

# 〇〇を混合正規分布でモデル化
mu = np.empty((K, M, D))
cov = np.empty((K, M, D, D))
model = np.empty(K, dtype=object)
for k in range(K):
    print(f'Class{k}')
    XX = XL[YL==k, :]
    model[k] = GaussianMixture(n_components=M, covariance_type='full')
    model[k].fit(XX)
    mu[k, ::] = model[k].means_
    cov[k, ::] = model[k].covariances_

# 各クラスの多変量正規分布の平均を可視化
Xm = np.empty((K, M, D))
for k in range(K):
    for m in range(M):
        Xm[k, m, :] = mu[k, m, :]
display(Xm.reshape((-1, D)), M, K)

In [ ]:
# 学習データの識別
N = XL.shape[0]
LL = np.empty((N, K))
for k in range(K):
    LL[:, k] = np.log(P[k]) + model[k].score_samples(XL)
Yp = np.argmax(LL, axis=1)
ncorrect = np.sum(YL == Yp)
print(f'学習データ: {ncorrect/N}')

# テストデータの識別
N = XT.shape[0]
LL = np.empty((N, K))
for k in range(K):
    LL[:, k] = np.log(P[k]) + model[k].score_samples(XT)
Yp = np.argmax(LL, axis=1)
ncorrect = np.sum(YT == Yp)
print(f'テストデータ: {ncorrect/N}')

#### Q4 上記を `M` の値をいろいろ変えながら実行し，結果を観察しなさい．

解答欄に，どのような実験結果が得られたか，および，それに対する考察を書きなさい．




【Q4 解答欄】

